In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline 

In [4]:
mapping=pd.read_csv('comments_employee_mapping.csv').dropna()
likes=pd.read_csv('comments_likeability.csv').dropna()
attr=pd.read_csv('employee_attrition.csv').dropna()
hap=pd.read_csv('happiness_level.csv').dropna()

attr['lastParticipationDate']=pd.to_datetime(attr['lastParticipationDate'],infer_datetime_format=True)
mapping['commentDate']=pd.to_datetime(mapping['commentDate'],infer_datetime_format=True)
hap['voteDate']=pd.to_datetime(hap['voteDate'],infer_datetime_format=True)

for d in [mapping,likes,attr,hap]:
    d['id']=d['employee'].map(str)+d['companyAlias']

In [121]:
#construct like matrix of size employees x comments
#1 denotes like, -1 denotes dislike

id_dict,comm_dict={},{}
for i in range(len(likes)):
    try: id_dict[likes['id'][i]]==0
    except: id_dict[likes['id'][i]]=len(id_dict.keys())
    try: comm_dict[likes['commentId'][i]]==0
    except: comm_dict[likes['commentId'][i]]=len(comm_dict.keys())
    if i%10000==0:print i
m=np.zeros([len(id_dict.keys()),len(comm_dict.keys())])
for i in range(len(likes)):
    m[id_dict[likes['id'][i]],comm_dict[likes['commentId'][i]]]= likes['liked'][i]*1 - likes['disliked'][i]*1
    if i%10000==0:print i
#np.save('like_matrix_sparse.npy',[scipy.sparse.coo_matrix(m),id_dict,comm_dict])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000


In [120]:
#construct comment matrix of size emp x comments
#1 denotes comment was made by that employee

id_dict_c,comm_dict_c={},{}
for i in range(len(mapping)):
    try: id_dict_c[mapping['id'][i]]==0
    except: id_dict_c[mapping['id'][i]]=len(id_dict_c.keys())
    try: comm_dict_c[mapping['commentId'][i]]==0
    except: comm_dict_c[mapping['commentId'][i]]=len(comm_dict_c.keys())
    if i%10000==0:print i
n=np.zeros([len(id_dict_c),len(comm_dict_c)])
for i in range(len(mapping)):
    n[id_dict_c[mapping['id'][i]],comm_dict_c[mapping['commentId'][i]]]=1
#np.save('comment_matrix_sparse.npy',[scipy.sparse.coo_matrix(n),id_dict_c,comm_dict_c])

0
10000
20000
30000
40000
50000
60000
70000


In [256]:
#ordering the like matrix to align with the attrition dataframe

l=[]
exc=0
for i in range(len(attr)):
    try:
        l.append(m[id_dict[attr['id'][i]],:])
    except:
        exc+=1
        l.append(np.zeros(len(comm_dict)))
l=np.vstack(l)
np.save('likes_matrix_sparse_arranged.npy',[scipy.sparse.coo_matrix(l),comm_dict])

In [257]:
#ordering the comments matrixx to align with the attrition dataframe

c=[]
exc_c=0
for i in range(len(attr)):
    try:
        c.append(n[id_dict_c[attr['id'][i]],:])
    except:
        exc_c+=1
        c.append(np.zeros(len(comm_dict_c)))
c=np.vstack(c)
np.save('comment_matrix_sparse_arranged.npy',[scipy.sparse.coo_matrix(c),comm_dict_c])